In [1]:
pip install openprompt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 146 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 18.3 MB/s 
     |████████████████████████████████| 346 kB 68.5 MB/s 
     |████████████████████████████████| 125 kB 76.1 MB/s 
     |████████████████████████████████| 4.2 MB 44.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
     |████████████████████████████████| 212 kB 56.2 MB/s 
     |████████████████████████████████| 1.1 MB 14.3 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 140 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 59.2 MB/s 
     |████████████████████████████████| 271 kB 48.1 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████

In [2]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Keypoints/train.csv")
dev = pd.read_csv("/content/drive/MyDrive/Keypoints/dev.csv")
test = pd.read_csv("/content/drive/MyDrive/Keypoints/test.csv")
train.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']
dev.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']
test.columns=['idx', 'topic', 'argument', 'key_point', 'stance', 'label']

for split in [train,test]:
  for i in split.index:
    arg = split['argument'][i]
    key = split['key_point'][i]
    if arg[-1] != '.':
      split.at[i, 'argument'] = arg + '.'
      split.at[i, 'key_point'] = key + '.'

    else:
      split.at[i, 'key_point'] = key + '.'

print(train[:5])

     idx                        topic  \
0  10137  We should ban human cloning   
1  10138  We should ban human cloning   
2  10139  We should ban human cloning   
3  10140  We should ban human cloning   
4  10141  We should ban human cloning   

                                            argument  \
0  a person created through cloning could potenti...   
1  a person created through cloning could potenti...   
2  a person created through cloning could potenti...   
3  a person created through cloning could potenti...   
4  Allowing humans to be cloned is setting the wo...   

                               key_point  stance  label  
0  Cloning is not understood enough yet.       1      0  
1   Cloning is unethical/anti-religious.       1      0  
2                  Cloning is unnatural.       1      0  
3                     Cloning is unsafe.       1      1  
4   Cloning is unethical/anti-religious.       1      0  


In [3]:
from openprompt.data_utils import InputExample
classes = ["matching","non-matching"]

dataset = {}

dataset['train'] = []
dataset['dev'] = []
dataset['test'] = []
for index, data in train.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['train'].append(input_example)
for index, data in dev.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['dev'].append(input_example)
for index, data in test.iterrows():
  input_example = InputExample(text_a = data['argument'], text_b = data['key_point'], label=int(data['label']), guid=data['idx'])
  dataset['test'].append(input_example)
  
print(dataset['train'][2])

{
  "guid": 10139,
  "label": 0,
  "meta": {},
  "text_a": "a person created through cloning could potentially have developmental problems caused by imperfections in the cloning process.",
  "text_b": "Cloning is unnatural.",
  "tgt_text": null
}



In [14]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
from openprompt.prompts import ManualTemplate
template_text = 'The argument: {"placeholder":"text_a"} and the Keypoint: {"placeholder":"text_b"} are {"mask"}.'
promptTemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

In [6]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    #label_words = {"matching","non-matching"}, # unsure
    tokenizer = tokenizer,
)

In [7]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

promptModel.cuda()

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): BertForMaskedLM(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
          

In [15]:
WrapperClass

openprompt.plms.mlm.MLMTokenizerWrapper

In [16]:
from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset = dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class = WrapperClass,
)

NotImplementedError: ignored

In [17]:
import torch

# making zero-shot inference using pretrained MLM with prompt
promptModel.eval()
with torch.no_grad():
    for batch in data_loader:
        logits = promptModel(batch)
        preds = torch.argmax(logits, dim = -1)
        print(classes[preds])
# predictions would be 1, 0 for classes 'matching', 'non-matching'

NameError: ignored